### 2016-01 Recuperatorio

Una red social almacena el contenido de los chats entre sus usuarios
en un RDD que tiene registros con el siguiente formato: (chat_id,
user_id, nickname, text). Queremos saber cuál es el usuario (user_id)
que mas preguntas hace en sus chats, contabilizamos una pregunta por
cada caracter “?” que aparezca en el campo text. Programar en Spark
un programa que identifique al usuario preguntón.

In [1]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [2]:
data = sc.parallelize(chats);

In [3]:
data.collect()

[(1, 1, 'damu', 'Qu\xc3\xa9 es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'S\xc3\xad! C\xc3\xb3mo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [4]:
data.map(lambda x: (x[1], x[3].count('?')))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if (x[1] > y[1]) else y)

(1, 3)

### 2016-01 Parcial

UBER almacena en un cluster todos los datos sobre el movimiento y
viajes de todos sus vehículos. Existe un proceso que nos devuelve un
RDD llamado trip_summary con los siguientes campos: (driver_id,
car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled),
Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para
Abril de 2016.

In [ ]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [ ]:
data = sc.parallelize(trips);

In [ ]:
data.collect()

In [ ]:
data.filter(lambda x: (x[4] > '20160400') and (x[4] < '20160500'))\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .map(lambda x: (x[0], x[1][1]/x[1][0]))\
    .reduce(lambda x, y: x if x[1] > y[1] else y)
    
# El último map no es necesario, agregado solo por claridad. 
# Sin el map lo único que cambia es la comparación que se hace en el reduce.

### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [ ]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [ ]:
data = sc.parallelize(stars);

In [ ]:
data.collect()

In [ ]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()
    
# El último map es simplemente para convertir la sequencia de spark en una lista de python para poder verla

### 2015-01 2do Recuperatorio

Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un
listado de tipo (patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos.

In [ ]:
textFile = sc.textFile("README.md")
words = textFile.flatMap(lambda line: line.split())
words.collect()

In [ ]:
def pattern(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    pat = ''
    found = ''
    for letter in word:
        if letter in found:
            pat += pat[found.index(letter)]
        else:
            found += letter
            if len(found) > len(letters):
                pat += '?'
            else:
                pat += letters[len(found)-1]
    return pat
   
words.map(lambda x: (pattern(x), 1))\
     .reduceByKey(lambda x, y: x+y)\
     .takeOrdered(10, lambda x: -x[1])

### 2016-02 Parcial

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [5]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [6]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [9]:
# generamos registros del tipo (word, docId) donde word sera nuestra key
words = documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])
words.collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [8]:
# llevamos scores a una representacion donde word sea la clave
scoresRDDForJoin = scoresRDD.map(lambda a: (a[1],(a[0],a[2])))
scoresRDDForJoin.collect()

[('pablo', ('thom', 1)),
 ('honey', ('thom', 1)),
 ('pablo', ('martin', -1)),
 ('honey', ('martin', -1)),
 ('ok', ('martin', 30)),
 ('computer', ('martin', 30))]

In [13]:
words_join_scores = words.join(scoresRDDForJoin)
words_join_scores.collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30)))]

In [14]:
# para poder realizar una acumulacion debemos llevar la informacion a la representacion ((docId, userId), score) 
words_join_scores_mapped = words_join_scores.map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))
words_join_scores_mapped.collect()

[((1, 'thom'), 1),
 ((1, 'martin'), -1),
 ((3, 'martin'), 30),
 ((1, 'thom'), 1),
 ((1, 'martin'), -1),
 ((3, 'martin'), 30)]

In [16]:
# acumulamos y llevamos a la representación pedida
words_join_scores_mapped.reduceByKey(lambda a,b: a + b)\
                        .map(lambda a: (a[0][0], a[0][1], a[1]))\
                        .collect()

[(3, 'martin', 60), (1, 'martin', -2), (1, 'thom', 2)]

### 2016-02 Recuperatorio

Contamos con un cluster que tiene 4 computadoras.

Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego eliminamos todos los mayores a 3 divisibles por 3 y nos queda (2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así sucesivamente.

El resultado final es una lista de números que son primos.

Programar este programa usando PySpark

#### Criterio

En primer lugar hay que crear un RDD con todos los números y hacer un parallelize con 4 particiones. 

Luego la resolución pasa por un ciclo en el cual en cada paso del ciclo se obtenga como pivote el mínimo número del RDD que sea mayor al número usado como pivote anteriormente, luego simplemente hay que filtrar el RDD eliminando los números que son múltiplos del pivote.

In [41]:
def is_multiple(pivot, value):
        if (value == pivot):
            return False
    
        if (value % pivot) == 0:
            return True
        else:
            return False

In [43]:
assert is_multiple(3, 6)
assert is_multiple(3, 9)
assert not is_multiple(6, 6)
assert not is_multiple(6, 3)
assert not is_multiple(3, 10)

In [44]:
# hacemos un caso de juguete de 2 a 2000
numbers = sc.parallelize(range(2,2000), 4)

pivot = 2

while ((pivot*pivot) <= 2000):
    new_pivot = numbers.filter(lambda a: a > pivot).reduce(lambda a, b: a if a < b else b)
    numbers = numbers.filter(lambda a: not is_multiple(pivot, a)).cache()
    pivot = new_pivot

In [45]:
numbers.collect()

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541,
 547,
 557,
 563,
 569,
 571,
 577,
 587,
 593,
 599,
 601,
 607,
 613,
 617,
 619,
 631,
 641,
 643,
 647,
 653,
 659,
 661,
 673,
 677,
 683,
 691,
 701,
 709,
 719,
 727,
 733,
 739,
 743,
 751,
 757,
 761,
 769,
 773,
 787,
 797,
 809,
 811,
 821,
 823,
 827,
 829,
 839,
 853,
 857,
 859,
 863,
 877,
 881,
 883,
 887,
 907,
 911,
 919,
 929,
 937,
 941,
 947,
 953,
 967,
 971,
 977,
 983,
 991,
 997,
 1009,
 1013,
 1019,


### 2017-01 Parcial

Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

#### Criterio

Existen diversas maneras de realizarlo, pero es posible resolverlo realizando un reduceByKey con clave (id_jugador,id_partido) acumulando la cantidad de asistencias por partido quedando ((id_jugador, id_partido), sum_asistencias).

Luego efectuamos un cambio de clave a id_jugador con un map para poder realizar un join de la informacion de la temporada usando id_jugador.

Una vez hecho esto, filtramos aquella informacion que cumple de mayor cantidad asistencias en el partido que el promedio de la temporada.

Dado que pueden quedarnos duplicados (un jugador podria superar su promedio de temporada en varios partidos), es importante al mapearla informacion al formato final realizar un distinct.

In [47]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

players_all_time_stats_rdd = sc.parallelize(players_all_time_stats)
scores_rdd = sc.parallelize(scores)

In [50]:
# cambio de clave a id_jugador, partido
scores_by_match = scores_rdd.map(lambda a: ((a[0],a[1]),a[3]))
scores_by_match = scores_by_match.reduceByKey(lambda a,b: a+b)
scores_by_match.collect()

[((1, 1), 200), ((1, 2), 300), ((3, 2), 100), ((2, 1), 300)]

In [51]:
# cambio de clave a id_jugador
scores_by_player = scores_by_match.map(lambda a: (a[0][0], a[1]))
scores_by_player.collect()

[(1, 200), (1, 300), (3, 100), (2, 300)]

In [53]:
# juntamos los datos para poder evaluar lo pedido
# preparamos la key del lado de informacion historica de jugadores.

all_time_with_key = players_all_time_stats_rdd.map(lambda a: (a[0],a))
all_time_with_key.collect()

[(1, (1, 'Manu Ginobili', 800)),
 (2, (2, 'Kobe Bryant', 100)),
 (3, (3, 'Marc Gasol', 25)),
 (4, (4, 'James Harden', 1000))]

In [54]:
scores_by_player = scores_by_player.join(all_time_with_key)
scores_by_player.collect()

[(1, (200, (1, 'Manu Ginobili', 800))),
 (1, (300, (1, 'Manu Ginobili', 800))),
 (2, (300, (2, 'Kobe Bryant', 100))),
 (3, (100, (3, 'Marc Gasol', 25)))]

In [63]:
overperformers = scores_by_player.filter(lambda a: a[1][0] > a[1][1][2])
overperformers.collect()

[(2, (300, (2, 'Kobe Bryant', 100))), (3, (100, (3, 'Marc Gasol', 25)))]

In [64]:
# veamos los duplicados
overperformers = overperformers.map(lambda a: a[1][1][1]).distinct()
overperformers.collect()

['Kobe Bryant', 'Marc Gasol']